In [2]:
import pandas as pd
import numpy as np
from random import shuffle
import copy
from dotenv import load_dotenv
import os
import ast
import openai
from openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS # in memory

from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain import OpenAI
from langchain.chains import ConversationChain
from langchain.chains import RetrievalQA
import pickle

load_dotenv()

class SudokuGenerator9X9:
	"""generates and solves Sudoku puzzles using a backtracking algorithm"""
	def __init__(self,grid=None):
		self.counter = 0
		#path is for the matplotlib animation
		self.path = []
		#if a grid/puzzle is passed in, make a copy and solve it
		if grid:
			if len(grid[0]) == 9 and len(grid) == 9:
				self.grid = grid
				self.original = copy.deepcopy(grid)
				self.solve_input_sudoku()
			else:
				print("input needs to be a 9x9 matrix")
		else:
			#if no puzzle is passed, generate one
			self.grid = [[0 for i in range(9)] for j in range(9)]
			self.generate_puzzle()
			self.original = copy.deepcopy(self.grid)
		
		
	def solve_input_sudoku(self):
		"""solves a puzzle"""
		self.generate_solution(self.grid)
		return

	def generate_puzzle(self):
		"""generates a new puzzle and solves it"""
		self.generate_solution(self.grid)
		self.print_grid('full solution')
		self.remove_numbers_from_grid()
		self.print_grid('with removed numbers')
		return

	def print_grid(self, grid_name=None):
		if grid_name:
			print(grid_name)
		for row in self.grid:
			print(row)
		return

	def test_sudoku(self,grid):
		"""tests each square to make sure it is a valid puzzle"""
		for row in range(9):
			for col in range(9):
				num = grid[row][col]
				#remove number from grid to test if it's valid
				grid[row][col] = 0
				if not self.valid_location(grid,row,col,num):
					return False
				else:
					#put number back in grid
					grid[row][col] = num
		return True

	def num_used_in_row(self,grid,row,number):
		"""returns True if the number has been used in that row"""
		if number in grid[row]:
			return True
		return False

	def num_used_in_column(self,grid,col,number):
		"""returns True if the number has been used in that column"""
		for i in range(9):
			if grid[i][col] == number:
				return True
		return False

	def num_used_in_subgrid(self,grid,row,col,number):
		"""returns True if the number has been used in that subgrid/box"""
		sub_row = (row // 3) * 3
		sub_col = (col // 3)  * 3
		for i in range(sub_row, (sub_row + 3)): 
			for j in range(sub_col, (sub_col + 3)): 
				if grid[i][j] == number: 
					return True
		return False

	def valid_location(self,grid,row,col,number):
		"""return False if the number has been used in the row, column or subgrid"""
		if self.num_used_in_row(grid, row,number):
			return False
		elif self.num_used_in_column(grid,col,number):
			return False
		elif self.num_used_in_subgrid(grid,row,col,number):
			return False
		return True

	def find_empty_square(self,grid):
		"""return the next empty square coordinates in the grid"""
		for i in range(9):
			for j in range(9):
				if grid[i][j] == 0:
					return (i,j)
		return

	def solve_puzzle(self, grid):
		"""solve the sudoku puzzle with backtracking"""
		for i in range(0,81):
			row=i//9
			col=i%9
			#find next empty cell
			if grid[row][col]==0:
				for number in range(1,10):
					#check that the number hasn't been used in the row/col/subgrid
					if self.valid_location(grid,row,col,number):
						grid[row][col]=number
						if not self.find_empty_square(grid):
							self.counter+=1
							break
						else:
							if self.solve_puzzle(grid):
								return True
				break
		grid[row][col]=0  
		return False

	def generate_solution(self, grid):
		"""generates a full solution with backtracking"""
		number_list = [1,2,3,4,5,6,7,8,9]
		for i in range(0,81):
			row=i//9
			col=i%9
			#find next empty cell
			if grid[row][col]==0:
				shuffle(number_list)      
				for number in number_list:
					if self.valid_location(grid,row,col,number):
						self.path.append((number,row,col))
						grid[row][col]=number
						if not self.find_empty_square(grid):
							return True
						else:
							if self.generate_solution(grid):
								#if the grid is full
								return True
				break
		grid[row][col]=0  
		return False

	def get_non_empty_squares(self,grid):
		"""returns a shuffled list of non-empty squares in the puzzle"""
		non_empty_squares = []
		for i in range(len(grid)):
			for j in range(len(grid)):
				if grid[i][j] != 0:
					non_empty_squares.append((i,j))
		shuffle(non_empty_squares)
		return non_empty_squares

	def remove_numbers_from_grid(self):
		"""remove numbers from the grid to create the puzzle"""
		#get all non-empty squares from the grid
		non_empty_squares = self.get_non_empty_squares(self.grid)
		non_empty_squares_count = len(non_empty_squares)
		rounds = 3
		while rounds > 0 and non_empty_squares_count >= 17:
			#there should be at least 17 clues
			row,col = non_empty_squares.pop()
			non_empty_squares_count -= 1
			#might need to put the square value back if there is more than one solution
			removed_square = self.grid[row][col]
			self.grid[row][col]=0
			#make a copy of the grid to solve
			grid_copy = copy.deepcopy(self.grid)
			#initialize solutions counter to zero
			self.counter=0      
			self.solve_puzzle(grid_copy)   
			#if there is more than one solution, put the last removed cell back into the grid
			if self.counter!=1:
				self.grid[row][col]=removed_square
				non_empty_squares_count += 1
				rounds -=1
		return

class SudokuGenerator4X4:
	"""generates and solves Sudoku puzzles using a backtracking algorithm"""
	def __init__(self,grid=None):
		self.counter = 0
		#path is for the matplotlib animation
		self.path = []
		#if a grid/puzzle is passed in, make a copy and solve it
		if grid:
			if len(grid[0]) == 4 and len(grid) == 4:
				self.grid = grid
				self.original = copy.deepcopy(grid)
				self.solve_input_sudoku()
			else:
				print("input needs to be a 4x4 matrix")
		else:
			#if no puzzle is passed, generate one
			self.grid = [[0 for i in range(4)] for j in range(4)]
			self.generate_puzzle()
			self.original = copy.deepcopy(self.grid)
		
		
	def solve_input_sudoku(self):
		"""solves a puzzle"""
		self.generate_solution(self.grid)
		return

	def generate_puzzle(self):
		"""generates a new puzzle and solves it"""
		self.generate_solution(self.grid)
		#self.print_grid('full solution')
		self.remove_numbers_from_grid()
		#self.print_grid('with removed numbers')
		return

	def print_grid(self, grid_name=None):
		if grid_name:
			print(grid_name)
		for row in self.grid:
			print(row)
		return

	def test_sudoku(self,grid):
		"""tests each square to make sure it is a valid puzzle"""
		for row in range(4):
			for col in range(4):
				num = grid[row][col]
				#remove number from grid to test if it's valid
				grid[row][col] = 0
				if not self.valid_location(grid,row,col,num):
					return False
				else:
					#put number back in grid
					grid[row][col] = num
		return True

	def num_used_in_row(self,grid,row,number):
		"""returns True if the number has been used in that row"""
		if number in grid[row]:
			return True
		return False

	def num_used_in_column(self,grid,col,number):
		"""returns True if the number has been used in that column"""
		for i in range(4):
			if grid[i][col] == number:
				return True
		return False

	def num_used_in_subgrid(self,grid,row,col,number):
		"""returns True if the number has been used in that subgrid/box"""
		sub_row = (row // 2) * 2
		sub_col = (col // 2)  * 2
		for i in range(sub_row, (sub_row + 2)): 
			for j in range(sub_col, (sub_col + 2)): 
				if grid[i][j] == number: 
					return True
		return False

	def valid_location(self,grid,row,col,number):
		"""return False if the number has been used in the row, column or subgrid"""
		if self.num_used_in_row(grid, row,number):
			return False
		elif self.num_used_in_column(grid,col,number):
			return False
		elif self.num_used_in_subgrid(grid,row,col,number):
			return False
		return True

	def find_empty_square(self,grid):
		"""return the next empty square coordinates in the grid"""
		for i in range(4):
			for j in range(4):
				if grid[i][j] == 0:
					return (i,j)
		return

	def solve_puzzle(self, grid):
		"""solve the sudoku puzzle with backtracking"""
		for i in range(0,16):
			row=i//4
			col=i%4
			#find next empty cell
			if grid[row][col]==0:
				for number in range(1,5):
					#check that the number hasn't been used in the row/col/subgrid
					if self.valid_location(grid,row,col,number):
						grid[row][col]=number
						if not self.find_empty_square(grid):
							self.counter+=1
							break
						else:
							if self.solve_puzzle(grid):
								return True
				break
		grid[row][col]=0  
		return False

	def generate_solution(self, grid):
		"""generates a full solution with backtracking"""
		number_list = [1,2,3,4]
		for i in range(0,16):
			row=i//4
			col=i%4
			#find next empty cell
			if grid[row][col]==0:
				shuffle(number_list)      
				for number in number_list:
					if self.valid_location(grid,row,col,number):
						self.path.append((number,row,col))
						grid[row][col]=number
						if not self.find_empty_square(grid):
							return True
						else:
							if self.generate_solution(grid):
								#if the grid is full
								return True
				break
		grid[row][col]=0  
		return False

	def get_non_empty_squares(self,grid):
		"""returns a shuffled list of non-empty squares in the puzzle"""
		non_empty_squares = []
		for i in range(len(grid)):
			for j in range(len(grid)):
				if grid[i][j] != 0:
					non_empty_squares.append((i,j))
		shuffle(non_empty_squares)
		return non_empty_squares

	def remove_numbers_from_grid(self):
		"""remove numbers from the grid to create the puzzle"""
		#get all non-empty squares from the grid
        # 
		non_empty_squares = self.get_non_empty_squares(self.grid)
		non_empty_squares_count = len(non_empty_squares)
		rounds = 3
		while rounds > 0 and non_empty_squares_count >= 4:
			#there should be at least 17 clues
			row,col = non_empty_squares.pop()
			non_empty_squares_count -= 1
			#might need to put the square value back if there is more than one solution
			removed_square = self.grid[row][col]
			self.grid[row][col]=0
			#make a copy of the grid to solve
			grid_copy = copy.deepcopy(self.grid)
			#initialize solutions counter to zero
			self.counter=0      
			self.solve_puzzle(grid_copy)   
			#if there is more than one solution, put the last removed cell back into the grid
			if self.counter!=1:
				self.grid[row][col]=removed_square
				non_empty_squares_count += 1
				rounds -=1
		return

In [3]:
def make_4X4_puzzles(i:int = 10):
    """
    Function that returns a dictionary
    with integeger keys 0 through i.
    The values are a tuple of two lists of lists.
    the first is the 4X4 puzzle unsolved grid,
    the second is the unique solution.
    """
    puzzle4X4_dict = dict()
    for j in range(i):
        new_puzzle = SudokuGenerator4X4()
        dfpuzzle4X4 = pd.DataFrame.from_records(new_puzzle.grid)
        new_puzzle_sol = SudokuGenerator4X4(grid=dfpuzzle4X4.to_numpy().tolist())
        puzzle4X4_dict[j] = (new_puzzle.grid, new_puzzle_sol.grid)
    return puzzle4X4_dict


def make_9X9_puzzles(i:int = 10):
    """
    Function that returns a dictionary
    with integeger keys 0 through i.
    The values are a tuple of two lists of lists.
    the first is the 4X4 puzzle unsolved grid,
    the second is the unique solution.
    """
    puzzle9X9_dict = dict()
    for j in range(i):
        new_puzzle = SudokuGenerator9X9()
        dfpuzzle9x9 = pd.DataFrame.from_records(new_puzzle.grid)
        new_puzzle_sol = SudokuGenerator9X9(grid=dfpuzzle9X9.to_numpy().tolist())
        puzzle9X9_dict[j] = (new_puzzle.grid, new_puzzle_sol.grid)
    return puzzle9X9_dict

In [5]:
puzzle4X4_dict = make_4X4_puzzles(i=100_000)

In [6]:

import pickle

with open("puzzle4X4_dict.pkl", "wb") as f:
    pickle.dump(puzzle4X4_dict, f)

combines sudokuexplorer (the chatbot type queries) and wittpdf (the retrievalqa to get the correct few shot examples)

In [7]:
texts = []

for i in puzzle4X4_dict.keys():
    example = str(puzzle4X4_dict[i][0]) + ' -> ' + str(puzzle4X4_dict[i][1])
    texts.append(example)

In [8]:
checks = []

for i in puzzle4X4_dict.keys():
    checks.append(str(puzzle4X4_dict[i][1]))

In [9]:
len(checks), len(set(checks)) # 4X4 Sudoku puzzles only have 288 distinct possible solutiions.

(100000, 288)

In [10]:
pd.Series(checks).value_counts()

[[1, 4, 3, 2], [2, 3, 1, 4], [4, 1, 2, 3], [3, 2, 4, 1]]    590
[[2, 4, 1, 3], [3, 1, 2, 4], [4, 2, 3, 1], [1, 3, 4, 2]]    586
[[3, 1, 4, 2], [2, 4, 3, 1], [1, 3, 2, 4], [4, 2, 1, 3]]    570
[[1, 2, 3, 4], [3, 4, 2, 1], [4, 3, 1, 2], [2, 1, 4, 3]]    566
[[4, 2, 1, 3], [3, 1, 4, 2], [1, 3, 2, 4], [2, 4, 3, 1]]    563
                                                           ... 
[[1, 3, 2, 4], [4, 2, 3, 1], [3, 4, 1, 2], [2, 1, 4, 3]]    232
[[2, 3, 4, 1], [4, 1, 2, 3], [3, 4, 1, 2], [1, 2, 3, 4]]    225
[[2, 3, 1, 4], [1, 4, 2, 3], [4, 1, 3, 2], [3, 2, 4, 1]]    223
[[1, 4, 3, 2], [3, 2, 1, 4], [2, 3, 4, 1], [4, 1, 2, 3]]    221
[[1, 2, 4, 3], [4, 3, 1, 2], [2, 1, 3, 4], [3, 4, 2, 1]]    220
Name: count, Length: 288, dtype: int64

In [11]:
len(texts)

100000

In [12]:
len(set(texts))

76962

In [13]:
sols = tuple((puzzle4X4_dict[i][1] for i in puzzle4X4_dict.keys()))

In [14]:
len(sols)

100000

# Making the embeddings

investigate using other embeddings, like the ones from HuggingFace

In [15]:
embeddings = OpenAIEmbeddings()

In [16]:
docsearch = FAISS.from_texts(texts, embeddings)

https://python.langchain.com/docs/integrations/vectorstores/faiss

In [19]:
docsearch.save_local("faiss_index")

In [20]:
docsearch = FAISS.load_local("faiss_index", embeddings)

In [21]:
with open("puzzle4X4_dict.pkl", "rb") as f:
    puzzle4X4_dict = pickle.load(f)

will end up pickling `docsearch` for the streamlit app eventually.

# RetrievalQA

In [22]:
from langchain.chains import RetrievalQA

# set up FAISS as a generic retriever
# make k an input parameter in the streamlit app
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":7})

# create the chain to answer question

rqa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name= 'gpt-4-1106-preview'),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [23]:
test_dict = make_4X4_puzzles(i=30)

In [24]:
puzzle = test_dict[0][0]
sol = test_dict[0][1]

In [25]:
puzzle

[[0, 0, 0, 1], [0, 1, 0, 0], [0, 2, 0, 0], [0, 4, 2, 0]]

First, use the rqa object to retrieve the most relevant example puzzles and their solutions.
This example list will then be fed into the ChatOpenAI model. A two step process.

In [26]:
results = rqa({"query":str(test_dict[0][0])})




In [27]:
type(results)
print(results.keys())
examples = results['source_documents']
print(type(examples))
examples[0].page_content

dict_keys(['query', 'result', 'source_documents'])
<class 'list'>


'[[0, 0, 0, 1], [4, 0, 0, 0], [2, 4, 0, 0], [0, 0, 2, 0]] -> [[3, 2, 4, 1], [4, 1, 3, 2], [2, 4, 1, 3], [1, 3, 2, 4]]'

In [28]:
def get_examples(new_puzzle:str):
    """
    takes in a new_puzzle with format like
    [[0, 0, 4, 0], [0, 0, 0, 0], [0, 3, 0, 0], [1, 0, 0, 2]]
    and returns the string of examples that will
    go into the template_string
    """
    results = rqa({"query":new_puzzle})
    examples = results['source_documents']
    eg_string = ''
    for eg in examples:
        eg_string += eg.page_content + '\n'
    return eg_string

In [42]:
result = rqa({"query": "what does the 4X4 sudoku puzzle" + str(test_dict[0][0]) + " have as solution? Think step by step. \
Don't fuck this up. Give me your best guess. look at the examples in the FAISS database. Do you understand the question? \
If not, what additional information do you need."})
print(result)

{'query': "what does the 4X4 sudoku puzzle[[0, 0, 0, 1], [0, 1, 0, 0], [0, 2, 0, 0], [0, 4, 2, 0]] have as solution? Think step by step. Don't fuck this up. Give me your best guess. look at the examples in the FAISS database. Do you understand the question? If not, what additional information do you need.", 'result': "Given the 4x4 Sudoku puzzle:\n\n```\n[[0, 0, 0, 1],\n[0, 1, 0, 0],\n[0, 2, 0, 0],\n[0, 4, 2, 0]]\n```\n\nLet's solve it step by step while respecting the rule that each row, column, and 2x2 subgrid must contain the numbers 1 through 4 without repetition.\n\n1. Start with the last column since it has the number 1 in the first row. We need to place 2, 3, and 4 in the remaining rows in some order. But since there's already a 2 in the third row, the last column must be `[1, 3, 4, 2]` from top to bottom.\n\n2. Now we have:\n\n```\n[[0, 0, 0, 1],\n[0, 1, 0, 3],\n[0, 2, 0, 4],\n[0, 4, 2, 2]]\n```\n\n3. In the second row, we have the numbers 1 and 3. We need to place 2 and 4. Sin

In [43]:
print(result['result'])

Given the 4x4 Sudoku puzzle:

```
[[0, 0, 0, 1],
[0, 1, 0, 0],
[0, 2, 0, 0],
[0, 4, 2, 0]]
```

Let's solve it step by step while respecting the rule that each row, column, and 2x2 subgrid must contain the numbers 1 through 4 without repetition.

1. Start with the last column since it has the number 1 in the first row. We need to place 2, 3, and 4 in the remaining rows in some order. But since there's already a 2 in the third row, the last column must be `[1, 3, 4, 2]` from top to bottom.

2. Now we have:

```
[[0, 0, 0, 1],
[0, 1, 0, 3],
[0, 2, 0, 4],
[0, 4, 2, 2]]
```

3. In the second row, we have the numbers 1 and 3. We need to place 2 and 4. Since 4 cannot be in the last column (it's already there in the third row), the second row must be `[4, 1, 2, 3]`.

4. We now have:

```
[[0, 0, 0, 1],
[4, 1, 2, 3],
[0, 2, 0, 4],
[0, 4, 2, 2]]
```

5. In the third column, we see that 2 is already in the fourth row, so we need to place the numbers 1 and 3 in the first and third rows. Given tha

In [44]:
print(results['source_documents'])

[Document(page_content='[[0, 0, 0, 1], [4, 0, 0, 0], [2, 4, 0, 0], [0, 0, 2, 0]] -> [[3, 2, 4, 1], [4, 1, 3, 2], [2, 4, 1, 3], [1, 3, 2, 4]]'), Document(page_content='[[0, 0, 0, 1], [1, 0, 0, 0], [0, 4, 0, 0], [0, 0, 2, 0]] -> [[4, 2, 3, 1], [1, 3, 4, 2], [2, 4, 1, 3], [3, 1, 2, 4]]'), Document(page_content='[[0, 0, 0, 1], [1, 0, 0, 0], [0, 4, 0, 0], [0, 0, 2, 0]] -> [[4, 2, 3, 1], [1, 3, 4, 2], [2, 4, 1, 3], [3, 1, 2, 4]]'), Document(page_content='[[0, 0, 0, 1], [1, 0, 0, 0], [0, 4, 0, 0], [0, 0, 2, 0]] -> [[4, 2, 3, 1], [1, 3, 4, 2], [2, 4, 1, 3], [3, 1, 2, 4]]'), Document(page_content='[[0, 0, 0, 1], [4, 0, 0, 0], [0, 4, 0, 0], [0, 0, 2, 0]] -> [[3, 2, 4, 1], [4, 1, 3, 2], [2, 4, 1, 3], [1, 3, 2, 4]]'), Document(page_content='[[0, 0, 0, 1], [2, 0, 0, 0], [0, 4, 0, 0], [0, 0, 1, 0]] -> [[4, 3, 2, 1], [2, 1, 4, 3], [1, 4, 3, 2], [3, 2, 1, 4]]'), Document(page_content='[[0, 0, 1, 0], [0, 0, 0, 2], [0, 0, 0, 0], [0, 1, 3, 0]] -> [[4, 2, 1, 3], [1, 3, 4, 2], [3, 4, 2, 1], [2, 1, 3, 4]]')

In [45]:
example_solutions = get_examples(str(test_dict[0][0])) # i = 0
print(example_solutions)

[[0, 0, 0, 1], [4, 0, 0, 0], [2, 4, 0, 0], [0, 0, 2, 0]] -> [[3, 2, 4, 1], [4, 1, 3, 2], [2, 4, 1, 3], [1, 3, 2, 4]]
[[0, 0, 0, 1], [1, 0, 0, 0], [0, 4, 0, 0], [0, 0, 2, 0]] -> [[4, 2, 3, 1], [1, 3, 4, 2], [2, 4, 1, 3], [3, 1, 2, 4]]
[[0, 0, 0, 1], [1, 0, 0, 0], [0, 4, 0, 0], [0, 0, 2, 0]] -> [[4, 2, 3, 1], [1, 3, 4, 2], [2, 4, 1, 3], [3, 1, 2, 4]]
[[0, 0, 0, 1], [1, 0, 0, 0], [0, 4, 0, 0], [0, 0, 2, 0]] -> [[4, 2, 3, 1], [1, 3, 4, 2], [2, 4, 1, 3], [3, 1, 2, 4]]
[[0, 0, 0, 1], [4, 0, 0, 0], [0, 4, 0, 0], [0, 0, 2, 0]] -> [[3, 2, 4, 1], [4, 1, 3, 2], [2, 4, 1, 3], [1, 3, 2, 4]]
[[0, 0, 0, 1], [2, 0, 0, 0], [0, 4, 0, 0], [0, 0, 1, 0]] -> [[4, 3, 2, 1], [2, 1, 4, 3], [1, 4, 3, 2], [3, 2, 1, 4]]
[[0, 0, 1, 0], [0, 0, 0, 2], [0, 0, 0, 0], [0, 1, 3, 0]] -> [[4, 2, 1, 3], [1, 3, 4, 2], [3, 4, 2, 1], [2, 1, 3, 4]]



In [46]:
chat_llm = ChatOpenAI(model_name = "gpt-4-1106-preview", temperature=0.2)
#chat_llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature=0)

puzzle_name_schema = ResponseSchema(name="input_puzzle",
                             description="This is the sudoku puzzle to be solved")

solution_schema = ResponseSchema(name="solution",
                                      description="This is the puzzle solution")

reasoning_schema = ResponseSchema(name="reasoning",
                                    description="This is the reasons for the solution")

confidence_schema = ResponseSchema(name="confidence",
                                   description="This is the confidence in the solution, a number between 0 and 1.")

response_schemas = [puzzle_name_schema,
                    solution_schema,
                    reasoning_schema,
                   confidence_schema]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()


In [52]:
template_string =  """
role: You An expert Sudoku Puzzle solver that can solve 9X9 puzzles as well as 4X4 puzzles.
content: Here are some example puzzles and their solutions.

{example_solutions}

The rules of 4x4 Sudoku puzzles are the same as with traditional Sudoku grids.
Only the number of cells and digits to be placed are different.
1. The numbers 1, 2, 3 and 4 must occur only once in each column
2. The numbers 1, 2, 3 and 4 must occur only once in each row.
3. The clues allocated at the beginning of the puzzle cannot be changed or moved.

Each row and column must contain each of the numbers 1-4. 
Each of the four corner areas of 4 cells must also contain each of the numbers 1-4.
In addition, the four corner cells must contain the numbers 1-4 AND
the central square of four cells must contain the numbers 1-4.


task: Using the rules of Sudoku, solve the initial 4X4 grid below. 0 indicates a missing
digit needing to be filled in. Think Step by Step.
Break it down carefully. Think logically and carefully. Each step in your solution
should be correct and obvious logically. No erasers needed for your expertise!
{puzzle}

The rules of 4x4 Sudoku puzzles are the same as with traditional Sudoku grids.
Only the number of cells and digits to be placed are different.
1. The numbers 1, 2, 3 and 4 must occur only once in each column
2. The numbers 1, 2, 3 and 4 must occur only once in each row.
3. The clues allocated at the beginning of the puzzle cannot be changed or moved.

Did you answer the previous question correctly? Your previous answer was {previous}. The actual answer was {answer}.
This is the puzzle to be solved:
{puzzle}

Work this out in a step-by-step way to be sure we have the right answer.

{format_instructions}
"""

In [53]:
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"input_puzzle": string  // This is the sudoku puzzle to be solved\n\t"solution": string  // This is the puzzle solution\n\t"reasoning": string  // This is the reasons for the solution\n\t"confidence": string  // This is the confidence in the solution, a number between 0 and 1.\n}\n```'

In [54]:
puzzle = test_dict[0][0]
puzzle

[[0, 0, 0, 1], [0, 1, 0, 0], [0, 2, 0, 0], [0, 4, 2, 0]]

In [55]:
previous = "correct"

In [56]:
print(template_string)


role: You An expert Sudoku Puzzle solver that can solve 9X9 puzzles as well as 4X4 puzzles.
content: Here are some example puzzles and their solutions.

{example_solutions}

The rules of 4x4 Sudoku puzzles are the same as with traditional Sudoku grids.
Only the number of cells and digits to be placed are different.
1. The numbers 1, 2, 3 and 4 must occur only once in each column
2. The numbers 1, 2, 3 and 4 must occur only once in each row.
3. The clues allocated at the beginning of the puzzle cannot be changed or moved.

Each row and column must contain each of the numbers 1-4. 
Each of the four corner areas of 4 cells must also contain each of the numbers 1-4.
In addition, the four corner cells must contain the numbers 1-4 AND
the central square of four cells must contain the numbers 1-4.


task: Using the rules of Sudoku, solve the initial 4X4 grid below. 0 indicates a missing
digit needing to be filled in. Think Step by Step.
Break it down carefully. Think logically and carefully.

In [57]:
prompt = ChatPromptTemplate.from_template(template=template_string)

In [58]:
example_solutions

'[[0, 0, 0, 1], [4, 0, 0, 0], [2, 4, 0, 0], [0, 0, 2, 0]] -> [[3, 2, 4, 1], [4, 1, 3, 2], [2, 4, 1, 3], [1, 3, 2, 4]]\n[[0, 0, 0, 1], [1, 0, 0, 0], [0, 4, 0, 0], [0, 0, 2, 0]] -> [[4, 2, 3, 1], [1, 3, 4, 2], [2, 4, 1, 3], [3, 1, 2, 4]]\n[[0, 0, 0, 1], [1, 0, 0, 0], [0, 4, 0, 0], [0, 0, 2, 0]] -> [[4, 2, 3, 1], [1, 3, 4, 2], [2, 4, 1, 3], [3, 1, 2, 4]]\n[[0, 0, 0, 1], [1, 0, 0, 0], [0, 4, 0, 0], [0, 0, 2, 0]] -> [[4, 2, 3, 1], [1, 3, 4, 2], [2, 4, 1, 3], [3, 1, 2, 4]]\n[[0, 0, 0, 1], [4, 0, 0, 0], [0, 4, 0, 0], [0, 0, 2, 0]] -> [[3, 2, 4, 1], [4, 1, 3, 2], [2, 4, 1, 3], [1, 3, 2, 4]]\n[[0, 0, 0, 1], [2, 0, 0, 0], [0, 4, 0, 0], [0, 0, 1, 0]] -> [[4, 3, 2, 1], [2, 1, 4, 3], [1, 4, 3, 2], [3, 2, 1, 4]]\n[[0, 0, 1, 0], [0, 0, 0, 2], [0, 0, 0, 0], [0, 1, 3, 0]] -> [[4, 2, 1, 3], [1, 3, 4, 2], [3, 4, 2, 1], [2, 1, 3, 4]]\n'

In [59]:
puzzle

[[0, 0, 0, 1], [0, 1, 0, 0], [0, 2, 0, 0], [0, 4, 2, 0]]

In [60]:
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"input_puzzle": string  // This is the sudoku puzzle to be solved\n\t"solution": string  // This is the puzzle solution\n\t"reasoning": string  // This is the reasons for the solution\n\t"confidence": string  // This is the confidence in the solution, a number between 0 and 1.\n}\n```'

```python
prompt.format_messages(example_solutions = example_solutions,
                       puzzle=puzzle,
                      previous='incorrect',
                      format_instructions=format_instructions)
```

In [61]:
print(template_string)


role: You An expert Sudoku Puzzle solver that can solve 9X9 puzzles as well as 4X4 puzzles.
content: Here are some example puzzles and their solutions.

{example_solutions}

The rules of 4x4 Sudoku puzzles are the same as with traditional Sudoku grids.
Only the number of cells and digits to be placed are different.
1. The numbers 1, 2, 3 and 4 must occur only once in each column
2. The numbers 1, 2, 3 and 4 must occur only once in each row.
3. The clues allocated at the beginning of the puzzle cannot be changed or moved.

Each row and column must contain each of the numbers 1-4. 
Each of the four corner areas of 4 cells must also contain each of the numbers 1-4.
In addition, the four corner cells must contain the numbers 1-4 AND
the central square of four cells must contain the numbers 1-4.


task: Using the rules of Sudoku, solve the initial 4X4 grid below. 0 indicates a missing
digit needing to be filled in. Think Step by Step.
Break it down carefully. Think logically and carefully.

In [75]:
prompt = ChatPromptTemplate.from_template(template=template_string)

def get_messages(i:int, format_instructions:str=format_instructions, previous:str='correct'):
    """
    i is the index of the test_dict solved puzzle dictionary.
    Returns the prompt.format_messages result
    where the puzzle to be solbed corresponds
    to the ith key of the puzzle dictionary.
    """
    messages = prompt.format_messages(
        example_solutions = get_examples(str(test_dict[i][0])), # i = 0
        puzzle = test_dict[i][0],
        previous=previous,
        answer=test_dict[i-1][1],
      #  input_puzzle=test_dict[i][0],
        format_instructions=format_instructions    
    )
    return messages

In [76]:
mess0 = get_messages(5)

In [77]:
print(mess0[0].content)


role: You An expert Sudoku Puzzle solver that can solve 9X9 puzzles as well as 4X4 puzzles.
content: Here are some example puzzles and their solutions.

[[0, 0, 4, 1], [0, 0, 0, 0], [0, 4, 0, 0], [0, 0, 2, 0]] -> [[3, 2, 4, 1], [4, 1, 3, 2], [2, 4, 1, 3], [1, 3, 2, 4]]
[[0, 4, 1, 0], [0, 0, 0, 0], [0, 0, 0, 3], [0, 0, 4, 0]] -> [[3, 4, 1, 2], [1, 2, 3, 4], [4, 1, 2, 3], [2, 3, 4, 1]]
[[0, 0, 0, 0], [4, 0, 0, 2], [0, 4, 1, 0], [0, 0, 0, 0]] -> [[3, 2, 4, 1], [4, 1, 3, 2], [2, 4, 1, 3], [1, 3, 2, 4]]
[[0, 0, 4, 2], [0, 2, 0, 0], [0, 0, 0, 1], [1, 0, 2, 0]] -> [[3, 1, 4, 2], [4, 2, 1, 3], [2, 4, 3, 1], [1, 3, 2, 4]]
[[0, 4, 1, 2], [0, 0, 0, 0], [0, 1, 0, 0], [0, 0, 4, 0]] -> [[3, 4, 1, 2], [1, 2, 3, 4], [4, 1, 2, 3], [2, 3, 4, 1]]
[[0, 0, 0, 0], [4, 0, 0, 2], [0, 4, 1, 0], [0, 0, 0, 0]] -> [[3, 2, 4, 1], [4, 1, 3, 2], [2, 4, 1, 3], [1, 3, 2, 4]]
[[0, 4, 1, 0], [0, 0, 0, 4], [0, 0, 0, 3], [0, 0, 4, 0]] -> [[3, 4, 1, 2], [1, 2, 3, 4], [4, 1, 2, 3], [2, 3, 4, 1]]


The rules of 4x4 Sudoku p

In [68]:
type(output_parser)

langchain.output_parsers.structured.StructuredOutputParser

# Add in memory

https://github.com/samwit/langchain-tutorials/blob/main/LC_Basics/YT_LangChain_Basic_Conversation_Chatbot_with_Memory_Demo.ipynb

In [97]:
template_string =  """
role: You An expert Sudoku Puzzle solver that can solve 9X9 puzzles as well as 4X4 puzzles.
content: Here are some example puzzles and their solutions.

{example_solutions}

The rules of 4x4 Sudoku puzzles are the same as with traditional Sudoku grids.
Only the number of cells and digits to be placed are different.
1. The numbers 1, 2, 3 and 4 must occur only once in each column
2. The numbers 1, 2, 3 and 4 must occur only once in each row.
3. The clues allocated at the beginning of the puzzle cannot be changed or moved.

Each row and column must contain each of the numbers 1-4. 
Each of the four corner areas of 4 cells must also contain each of the numbers 1-4.
In addition, the four corner cells must contain the numbers 1-4 AND
the central square of four cells must contain the numbers 1-4.


task: Using the rules of Sudoku, solve the initial 4X4 grid below. 0 indicates a missing
digit needing to be filled in. Think Step by Step.
Break it down carefully. Think logically and carefully. Each step in your solution
should be correct and obvious logically. No erasers needed for your expertise!
{puzzle}

The rules of 4x4 Sudoku puzzles are the same as with traditional Sudoku grids.
Only the number of cells and digits to be placed are different.
1. The numbers 1, 2, 3 and 4 must occur only once in each column
2. The numbers 1, 2, 3 and 4 must occur only once in each row.
3. The clues allocated at the beginning of the puzzle cannot be changed or moved.

Did you answer the previous question correctly? Your previous answer was {previous}. The actual solution is {answer}.
This is the puzzle to be solved:
{puzzle}

Work this out in a step-by-step way. Take your time.

{format_instructions}
"""

In [98]:
prompt = ChatPromptTemplate.from_template(template=template_string)

def get_messages(i:int, format_instructions:str=format_instructions, previous:str='correct'):
    """
    i is the index of the test_dict solved puzzle dictionary.
    Returns the prompt.format_messages result
    where the puzzle to be solbed corresponds
    to the ith key of the puzzle dictionary.
    """
    messages = prompt.format_messages(
        example_solutions = get_examples(str(test_dict[i][0])), # i = 0
        puzzle = test_dict[i][0],
        previous=previous,
        answer=test_dict[i-1],
      #  input_puzzle=test_dict[i][0],
        format_instructions=format_instructions    
    )
    return messages

In [99]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain import OpenAI
from langchain.chains import ConversationChain

In [100]:
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler 

In [101]:
llm_llama2 = Ollama(model="llama2", 
            #  callback_manager = CallbackManager([StreamingStdOutCallbackHandler()]),
            temperature=0.9,
             )

In [102]:
llm = ChatOpenAI(model_name = "gpt-4-1106-preview", temperature=0)
#llm = ChatOpenAI(model_name = "gpt-4", temperature=0)
# we set a low k=2, to only keep the last 2 interactions in memory
#llm = ChatOpenAI(model_name = "davinci-002", temperature=0.2)
window_memory = ConversationBufferWindowMemory(k=2)

conversation = ConversationChain(
    llm=llm, 
   # llm=llm_llama2,
    verbose=True, 
    memory=window_memory
)

In [103]:
results = []
for i in range(19, 30):
    if len(results) > 0:
        last = results[-1]
        if last == True:
            previous = 'correct'
        else:
            previous = 'incorrect'
    else:
        previous = 'correct'
   # example_solutions = get_examples(str(test_dict[i][0])) # i = 0
    messages =  get_messages(i=i, previous=previous)
    response = conversation.predict(input=messages[0].content)
    try:
        response_as_dict = output_parser.parse(response)
        res = test_dict[i][1] == ast.literal_eval(response_as_dict["solution"])
        print(test_dict[i][1], ast.literal_eval(response_as_dict["solution"]))
        print(res, i, "can use output_parser", response_as_dict["confidence"])
        results.append(res)
    except:
        res = str(test_dict[i][1])  in response
        #print(puzzle4X4_dict[i][1], ast.literal_eval(response_as_dict["solution"]))
        print(response)
        print(res, i, "can not use output_parser")
        results.append(res)
    print(pd.Series(results).value_counts())

print(pd.Series(results).value_counts())



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 
role: You An expert Sudoku Puzzle solver that can solve 9X9 puzzles as well as 4X4 puzzles.
content: Here are some example puzzles and their solutions.

[[0, 0, 4, 1], [0, 0, 0, 0], [0, 4, 0, 0], [0, 0, 2, 0]] -> [[3, 2, 4, 1], [4, 1, 3, 2], [2, 4, 1, 3], [1, 3, 2, 4]]
[[0, 0, 0, 0], [4, 0, 0, 2], [0, 4, 1, 0], [0, 0, 0, 0]] -> [[3, 2, 4, 1], [4, 1, 3, 2], [2, 4, 1, 3], [1, 3, 2, 4]]
[[0, 0, 0, 0], [4, 0, 0, 2], [0, 4, 1, 0], [0, 0, 0, 0]] -> [[3, 2, 4, 1], [4, 1, 3, 2], [2, 4, 1, 3], [1, 3, 2, 4]]
[[0, 0, 0, 0], [4, 0, 0, 2], [0, 4, 1, 0], [1, 0, 0, 0]] -> [[3, 2, 4, 1], [4, 1, 3, 2], [2, 4, 1, 3], [1, 3, 2, 4]]
[[0, 0, 0, 0], [4, 0, 0, 2], [0, 4, 0, 0], [1, 

In [79]:
test_dict[0][0]

[[0, 2, 0, 0], [0, 0, 0, 3], [4, 0, 3, 0], [0, 0, 0, 4]]

In [81]:
test_dict[0][1]

[[3, 2, 4, 1], [1, 4, 2, 3], [4, 1, 3, 2], [2, 3, 1, 4]]

In [80]:
rqa({"query":str(test_dict[0][0])})

{'query': '[[0, 2, 0, 0], [0, 0, 0, 3], [4, 0, 3, 0], [0, 0, 0, 4]]',
 'result': '[[3, 2, 1, 4], [2, 4, 3, 1], [4, 3, 2, 1], [1, 3, 4, 2]]',
 'source_documents': [Document(page_content='[[0, 0, 0, 3], [1, 0, 0, 0], [0, 0, 2, 0], [0, 1, 0, 0]] -> [[4, 2, 1, 3], [1, 3, 4, 2], [3, 4, 2, 1], [2, 1, 3, 4]]'),
  Document(page_content='[[0, 0, 3, 1], [0, 0, 0, 0], [0, 3, 0, 0], [0, 0, 2, 0]] -> [[4, 2, 3, 1], [3, 1, 4, 2], [2, 3, 1, 4], [1, 4, 2, 3]]'),
  Document(page_content='[[0, 0, 4, 0], [0, 1, 0, 0], [0, 0, 0, 3], [0, 0, 2, 0]] -> [[3, 2, 4, 1], [4, 1, 3, 2], [2, 4, 1, 3], [1, 3, 2, 4]]'),
  Document(page_content='[[0, 0, 3, 0], [0, 2, 0, 0], [0, 0, 0, 0], [1, 3, 0, 0]] -> [[4, 1, 3, 2], [3, 2, 4, 1], [2, 4, 1, 3], [1, 3, 2, 4]]'),
  Document(page_content='[[0, 0, 0, 0], [0, 2, 3, 0], [0, 3, 0, 0], [4, 0, 2, 0]] -> [[3, 4, 1, 2], [1, 2, 3, 4], [2, 3, 4, 1], [4, 1, 2, 3]]'),
  Document(page_content='[[0, 0, 0, 3], [4, 0, 2, 0], [0, 4, 3, 0], [0, 0, 0, 0]] -> [[2, 1, 4, 3], [4, 3, 2, 1], 